# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; William Voigt.</p>
### <p style="text-align: right;"> &#9989; voigtwil</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [6]:
#Write your code here
import pandas as pd

aussie_weather_data = pd.read_csv("aussie_weather_data")



#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [7]:
#Write your code here

specific_cities = []

for i in aussie_weather_data["Location"]:
    if i not in specific_cities:
        specific_cities.append(i)
    
specific_cities

['NorfolkIsland',
 'SydneyAirport',
 'WaggaWagga',
 'MelbourneAirport',
 'Mildura',
 'Watsonia',
 'Brisbane',
 'Cairns',
 'Townsville',
 'MountGambier',
 'Nuriootpa',
 'PerthAirport',
 'Perth',
 'AliceSprings',
 'Darwin']

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [8]:
#Write your code here

mask = aussie_weather_data["Location"] == "WaggaWagga"

WaggaWagga_weather_data = aussie_weather_data[mask]

WaggaWagga_weather_data

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
5334,2009,1,1,WaggaWagga,9.9,28.4,0.0,9.8,13.5,157.5,...,37.0,24.0,1005.2,1003.5,1.0,1.0,20.8,27.0,0,0
5335,2009,1,2,WaggaWagga,8.9,25.3,0.0,14.8,13.7,202.5,...,37.0,10.0,1013.9,1012.9,1.0,0.0,14.7,23.2,0,0
5336,2009,1,3,WaggaWagga,11.0,30.7,0.0,9.8,13.7,67.5,...,43.0,18.0,1017.8,1014.3,0.0,1.0,18.9,28.0,0,0
5337,2009,1,4,WaggaWagga,14.7,35.2,0.0,9.4,12.1,90.0,...,46.0,19.0,1014.8,1010.6,1.0,4.0,24.7,33.7,0,0
5338,2009,1,5,WaggaWagga,11.8,36.5,0.0,11.6,13.4,180.0,...,23.0,5.0,1013.1,1009.8,1.0,1.0,23.6,34.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,2016,4,22,WaggaWagga,14.7,24.2,1.4,2.8,7.9,225.0,...,85.0,52.0,1021.7,1018.9,1.0,3.0,17.5,22.0,1,0
7746,2016,4,23,WaggaWagga,9.6,23.5,0.0,3.4,10.7,67.5,...,56.0,24.0,1026.2,1024.2,1.0,1.0,15.7,22.5,0,0
7747,2016,4,24,WaggaWagga,7.5,24.1,0.0,3.6,10.7,22.5,...,48.0,25.0,1030.6,1027.6,0.0,1.0,15.0,23.7,0,0
7748,2016,4,25,WaggaWagga,8.0,24.2,0.0,3.8,10.4,22.5,...,63.0,32.0,1032.0,1027.8,0.0,1.0,16.0,23.7,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [9]:
#Write your code here
import sklearn
from sklearn.model_selection import train_test_split

X = WaggaWagga_weather_data.iloc[:, 4:24]

y= WaggaWagga_weather_data["RainTomorrow"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33, random_state=69)



## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

*A logistic regression would make the most sense since whether or not it rains tomorrow is a binary variable.*

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [14]:
#Write your code here


from sklearn.metrics import accuracy_score
import statsmodels.api as sm

logreg = sm.Logit(y_train,X_train)

result = logreg.fit()

print(result.summary() )

      






Optimization terminated successfully.
         Current function value: 0.224277
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 1618
Model:                          Logit   Df Residuals:                     1598
Method:                           MLE   Df Model:                           19
Date:                Sat, 22 Apr 2023   Pseudo R-squ.:                  0.5122
Time:                        18:13:54   Log-Likelihood:                -362.88
converged:                       True   LL-Null:                       -743.84
Covariance Type:            nonrobust   LLR p-value:                2.650e-149
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
MinTemp           0.0093      0.056      0.166      0.868      -0.101       0.119
MaxTemp           0.

Statistically Significant Variables:

Sunshine
WindGustSpeed
WindSpeed9am
Humidity3pm
Pressure9am
Pressure3pm


#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [22]:
#Write your code here
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

predictions = result.predict(X_test)
binary_predictions = (predictions > 0.5).astype(int)
cm = confusion_matrix(y_true=y_test, y_pred=binary_predictions)

precision = precision_score(y_true=y_test, y_pred=binary_predictions)
recall = recall_score(y_true=y_test, y_pred=binary_predictions)

print("Precision: ", precision)
print("Recall: ", recall)
cm

Precision:  0.7377049180327869
Recall:  0.6081081081081081


array([[618,  32],
       [ 58,  90]])

#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

*Precision is higher than recall. In other words, the model is making fewer false positive predictions compared to false negatives. It performs better the circumstances where the data contains a lot negatives.*

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [41]:
#Write your code for calculating the frequency of rain here.

count = 0

for i in WaggaWagga_weather_data["RainTomorrow"]:
    if i == 0:
        count+= 1


count / len(WaggaWagga_weather_data["RainTomorrow"])


0.8232615894039735

*82.3% of the time.*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [ ]:
#Write your code here

*Write down your best fir hyper-parameters here.*

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [ ]:
#Write your code here

## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


*Write down your response here.*

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


*Write down the accuracy, precision, recall, and seed number for each of your iterations.*

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

*Write down your response here.*

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

*Write down your response here.*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University